In [275]:
import numpy as np # linear algebra, data manipulation
import pandas as pd # data processing, # Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
# change to  dates and time default formats
from datetime import datetime
import time

In [276]:
train_data= pd.read_csv('C:\\Users\\HP\\Desktop\\train_users_2.csv')
test_data= pd.read_csv('C:\\Users\\HP\\Desktop\\test_users.csv')

In [277]:
train_data.loc[(train_data.age > 95) | (train_data.age < 15), 'age'] = np.nan
train_data['age']= train_data['age'].fillna(train_data['age'].median())
#I choose to fill with median because it's robust with outliers
train_data.age.isnull().sum()#checking for null values after filling

0

In [278]:
train_data.dropna(inplace=True) #The dropna () method removes the rows that contains NULL values

In [279]:
train_data=train_data.drop_duplicates() #used to remove duplicate values
train_data =train_data.drop(['id'],axis=1) #removing id because it is not useful in our dataset , values are very large
# Converting to date time format
train_data['date_account_created'] = pd.to_datetime((train_data['date_account_created']))
train_data['date_first_booking'] = pd.to_datetime(train_data['date_first_booking'])
train_data['timestamp_first_active'] = pd.to_datetime(train_data['timestamp_first_active'])
train_data.head()

,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
2,2010-09-28,1970-01-01 05:34:50.600,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,2011-12-05,1970-01-01 05:34:51.000,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,2010-09-14,1970-01-01 05:34:51.200,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
5,2010-01-01,1970-01-01 05:35:00.100,2010-01-02,-unknown-,34.0,basic,0,en,other,other,omg,Web,Mac Desktop,Chrome,US
6,2010-01-02,1970-01-01 05:35:00.100,2010-01-05,FEMALE,46.0,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US


    Using decision Tree Classifier

In [280]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

train_data.loc[train_data.gender=='MALE', 'gender'] = 0
train_data.loc[train_data.gender=='FEMALE', 'gender'] = 1
train_data.loc[train_data.gender=='-unknown-', 'gender'] = 2
train_data.loc[train_data.gender=='OTHER', 'gender'] = 3
train_data['date_account_created'] = train_data['date_account_created'].apply(pd.Timestamp.timestamp)
train_data['date_first_booking'] = train_data['date_first_booking'].apply(pd.Timestamp.timestamp)
train_data['timestamp_first_active'] = train_data['timestamp_first_active'].apply(pd.Timestamp.timestamp)

train_data['signup_method'].replace(['basic','facebook','google'],[0, 1,2], inplace=True)
#train_data=pd.get_dummies(train_data,columns=['language','affiliate_channel','affiliate_provider','first_affiliate_tracked','signup_app','first_device_type','first_browser','country_destination'])
feature_cols=['gender','age','date_account_created','signup_flow']
#df1=pd.DataFrame(data=train_data.values, columns=train_data.columns)
# Scale data before applying PCA
#scaling=StandardScaler()
# Use fit and transform method
#scaling.fit(df1)
#Scaled_data=scaling.transform(df1)
# Set the n_components=3
#principal=PCA(n_components=3)
#principal.fit(Scaled_data)
#x=principal.transform(Scaled_data)
# Check the dimensions of data after PCA
#print(x.shape)
x=train_data[feature_cols]
y=train_data.country_destination
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1) # 70% training and 30% test


In [281]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [282]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA(n_components=1)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

In [283]:
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [284]:
#Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.540705900347979


Using KNN Classifier

In [285]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
model=KNeighborsClassifier(n_neighbors=17)
model.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=17)

In [286]:
# Accuracy on Tesing Data
y_pred=model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6991319643608275


Using Naive Bayes

In [287]:
from sklearn.naive_bayes import GaussianNB
NB=GaussianNB()
NB.fit(X_train,y_train)
y_pred_NB = clf.predict(X_test)
Acc=NB.score(X_test, y_pred)
print(Acc)

0.9990440136132461
